# NT + ESM-2 ZERO-SHOT Ver


### 1) Split parquet into train/val/test (chr20/21 as test)


In [1]:
from split_data import main as split_main
from config import RAW_PARQUET
split_main(RAW_PARQUET)

Saved:
  train: 114568 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\train.parquet
  val  : 20219 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\val.parquet
  test : 4562 -> d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\test.parquet


### 2) Precompute embeddings (NT + ESM-2)


In [2]:
%env TOKENIZERS_PARALLELISM=false
from precompute_embeddings import main as emb_main
emb_main()

env: TOKENIZERS_PARALLELISM=false


c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
Loading NT...


Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading ESM-2...


...

[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\train_embeddings.pt
Embedding split d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\val.parquet (20219 rows)


100%|██████████| 5055/5055 [02:56<00:00, 28.71it/s]


[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\val_embeddings.pt
Embedding split d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\data\test.parquet (4562 rows)


100%|██████████| 1141/1141 [00:39<00:00, 28.68it/s]


[saved] d:\Biosequence\Bio_sequence_Research_AITALAB-main\train\task3_variant_prediction\embeddings\test_embeddings.pt


### 3) Train fusion classifier

Để theo dõi tiến độ train trong tensorboard:

- Mở terminal -> nhập: tensorboard --logdir train/task3_variant_prediction/runs -> truy cập vào link được in ra


In [1]:
from train import train
from config import LR, EPOCHS, BATCH_SIZE, PATIENCE, DROPOUT, SEED, PROJ_DIM, FUSION_HIDDEN, WEIGHT_DECAY
import argparse

parser = argparse.ArgumentParser(description="Train fusion classifier with enhanced features.")
parser.add_argument("--proj_dim", type=int, default=PROJ_DIM)
parser.add_argument("--fusion_hidden", type=list, default=FUSION_HIDDEN)
parser.add_argument("--lr", type=float, default=1e-4, help="Learning rate")
parser.add_argument("--epochs", type=int, default=EPOCHS)
parser.add_argument("--batch_size", type=int, default=BATCH_SIZE)
parser.add_argument("--patience", type=int, default=PATIENCE)
parser.add_argument("--dropout", type=float, default=0.3)
parser.add_argument("--weight_decay", type=float, default=WEIGHT_DECAY, help="L2 regularization lambda")
parser.add_argument("--seed", type=int, default=SEED)
parser.add_argument("--exp_name", type=str, default="experiment_2", help="Experiment name (auto-generated if not provided)")
parser.add_argument("--log_dir", type=str, default=None, help="TensorBoard log directory (auto: runs/exp_name)")

args = parser.parse_args([])

result = train(args)
result

c:\Users\Dung\anaconda3\envs\stable\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TRAINING CONFIGURATION:
  Experiment Name: experiment_2
  Device: cuda
  Learning Rate: 0.0001
  Epochs: 30
  Batch Size: 128
  Patience: 5
  Dropout: 0.3
  Seed: 42
  Proj Dim: 512
  Fusion Hidden: [512, 256]
  Log Dir: runs\experiment_2
  Experiment Dir: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_2


[1] Train Loss: 0.1802 | Val Loss: 0.1539 | Train Acc: 0.9306 | Val Acc: 0.9396
--> Saved best model checkpoint to d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\best_fusion_model.pt


[2] Train Loss: 0.1386 | Val Loss: 0.1491 | Train Acc: 0.9467 | Val Acc: 0.9425
--> Saved best model checkpoint to d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\best_fusion_model.pt


[3] Train Loss: 0.1198 | Val Loss: 0.1560 | Train Acc: 0.9538 | Val Acc: 0.9410


...

[5] Train Loss: 0.0848 | Val Loss: 0.1723 | Train Acc: 0.9681 | Val Acc: 0.9406


[6] Train Loss: 0.0686 | Val Loss: 0.1897 | Train Acc: 0.9751 | Val Acc: 0.9417


[7] Train Loss: 0.0432 | Val Loss: 0.2519 | Train Acc: 0.9845 | Val Acc: 0.9412
Early stopping triggered.

--- Testing with Best Model ---


[TEST] Loss: 0.1584 | AUC: 0.9363 | MCC: 0.8726 | Acc: 0.9375
[TEST] Balanced Acc: 0.9363 | F1_macro: 0.9363 | Precision: 0.9273 | Recall: 0.9278

✓ Experiment saved to: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_2
  - Config: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_2\config.json
  - Args: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_2\args.json
  - Results: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_2\results.json
  - Model: d:\Bio_sequence_Research_AITALAB\train\task3_variant_prediction\embeddings\experiments\experiment_2\best_model.pt


{'acc': 0.9375274181365967,
 'auc': 0.9363436102867126,
 'balanced_acc': 0.9363436698913574,
 'f1_macro': 0.936316728591919,
 'mcc': 0.8726336359977722,
 'precision': 0.9273004531860352,
 'recall': 0.9277721047401428,
 'loss': 0.15842028631731142}

### 4) Xem lại kết quả experiments

Các experiments được lưu tự động trong `embeddings/experiments/<exp_name>/`


In [2]:
# Xem danh sách tất cả experiments
import os
import json
import pandas as pd

exp_base_dir = "embeddings/experiments"
if os.path.exists(exp_base_dir):
    experiments = []
    for exp_name in os.listdir(exp_base_dir):
        exp_dir = os.path.join(exp_base_dir, exp_name)
        results_file = os.path.join(exp_dir, "results.json")
        if os.path.exists(results_file):
            with open(results_file, "r") as f:
                data = json.load(f)
                experiments.append({
                    "exp_name": exp_name,
                    "timestamp": data.get("timestamp", ""),
                    "test_auc": data["test_results"].get("auc", 0),
                    "test_acc": data["test_results"].get("acc", 0),
                    "test_mcc": data["test_results"].get("mcc", 0),
                    "test_balanced_acc": data["test_results"].get("balanced_acc", 0),
                    "test_f1_macro": data["test_results"].get("f1_macro", 0),
                    "test_precision": data["test_results"].get("precision", 0),
                    "test_recall": data["test_results"].get("recall", 0),
                    "best_val_loss": data.get("best_val_loss", 0),
                    "epochs": data.get("epochs_trained", 0),
                })
    
    if experiments:
        df = pd.DataFrame(experiments)
        df = df.sort_values("timestamp", ascending=False)
        print("Danh sách experiments:")
        print(df.to_string(index=False))
    else:
        print("Chưa có experiments nào.")
else:
    print(f"Thư mục {exp_base_dir} chưa tồn tại.")

Danh sách experiments:
    exp_name                  timestamp  test_auc  test_acc  test_mcc  test_balanced_acc  test_f1_macro  test_precision  test_recall  best_val_loss  epochs
experiment_2 2025-12-26T22:01:18.202990  0.936344  0.937527  0.872634           0.936344       0.936317          0.9273     0.927772       0.149093       7


In [3]:
# Xem chi tiết một experiment cụ thể
exp_name = "experiment_2"  # Thay bằng exp_name muốn xem

exp_dir = f"embeddings/experiments/{exp_name}"
if os.path.exists(exp_dir):
    print(f"=== Experiment: {exp_name} ===\n")
    
    # Xem config
    config_file = os.path.join(exp_dir, "config.json")
    if os.path.exists(config_file):
        with open(config_file, "r") as f:
            config = json.load(f)
        print("Config:")
        for k, v in config.items():
            print(f"  {k}: {v}")
        print()
    
    # Xem results
    results_file = os.path.join(exp_dir, "results.json")
    if os.path.exists(results_file):
        with open(results_file, "r") as f:
            results = json.load(f)
        print("Results:")
        print(f"  Best Val Loss: {results.get('best_val_loss', 0):.4f}")
        print(f"  Test AUC: {results['test_results'].get('auc', 0):.4f}")
        print(f"  Test Acc: {results['test_results'].get('acc', 0):.4f}")
        print(f"  Test MCC: {results['test_results'].get('mcc', 0):.4f}")
        print(f"  Test Balanced Acc: {results['test_results'].get('balanced_acc', 0):.4f}")
        print(f"  Test F1_macro: {results['test_results'].get('f1_macro', 0):.4f}")
        print(f"  Test Precision: {results['test_results'].get('precision', 0):.4f}")
        print(f"  Test Recall: {results['test_results'].get('recall', 0):.4f}")
        print(f"  Epochs Trained: {results.get('epochs_trained', 0)}")
else:
    print(f"Experiment '{exp_name}' không tồn tại.")

=== Experiment: experiment_2 ===

Config:
  exp_name: experiment_2
  timestamp: 2025-12-26T21:59:59.432663
  lr: 0.0001
  epochs: 30
  batch_size: 128
  patience: 5
  dropout: 0.3
  seed: 42
  proj_dim: 512
  fusion_hidden: [512, 256]
  log_dir: runs\experiment_2

Results:
  Best Val Loss: 0.1491
  Test AUC: 0.9363
  Test Acc: 0.9375
  Test MCC: 0.8726
  Test Balanced Acc: 0.9363
  Test F1_macro: 0.9363
  Test Precision: 0.9273
  Test Recall: 0.9278
  Epochs Trained: 7


# NT + ESM-2 FINE-TUNED Ver
